# [example](https://blog.ruanbekker.com/blog/2018/04/23/using-paramiko-module-in-python-to-execute-remote-bash-commands/)

In [115]:
import os
    
def hello():
    print("fh")
    print(os.getpid())

In [121]:
# https://stackoverflow.com/questions/4858100/how-to-list-imported-modules
import sys
modulenames = set(sys.modules) & set(globals())
allmodules = [sys.modules[name] for name in modulenames]
allmodules

[<module 'os' from '/Users/vincenttang/anaconda3/lib/python3.7/os.py'>,
 <module 'sys' (built-in)>]

In [60]:
# for i in inspect.getsourcelines(hello):
source_code = inspect.getsource(hello)
source_code

'def hello():\n    print("fh")\n    print(os.getpid())\n'

In [83]:
import os

def hello():
    print("fh")
    print(os.getpid())

In [89]:
b = 'if __name__ == "__main__":\n     import os\n     hello()\n'

In [90]:
c = source_code + b

In [91]:
# https://thepythonguru.com/pickling-objects-in-python/
f = open("demofile2.py", "w")  # a for append, w for overwrite
f.write(c)
f.close()

In [95]:
import paramiko

ssh = paramiko.SSHClient()  # https://blog.ruanbekker.com/blog/2018/04/23/using-paramiko-module-in-python-to-execute-remote-bash-commands/
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname='3.94.21.39', username='ubuntu', key_filename='test.pem')


ftp_client=ssh.open_sftp()  # https://medium.com/@keagileageek/paramiko-how-to-ssh-and-file-transfers-with-python-75766179de73
ftp_client.put('demofile2.py', "demofile2.py")
ftp_client.close()

In [99]:
ssh.connect(hostname='3.94.21.39', username='ubuntu', key_filename='test.pem')

stdin, stdout, stderr = ssh.exec_command("python3 demofile2.py")

for line in stdout.read().splitlines():
    print(line)

ssh.close()


b'fh'
b'3040'


In [189]:
import paramiko

def my_decorator(func):
    def modfied_func():
        source_code = inspect.getsource(func).splitlines()[1:]
        
        f = open("temp.py", "w")  # a for append, w for overwrite
        for l in ls:
            f.write(l + "\n")
        f.close()
        
        f = open("temp.py", "a")  # a for append, w for overwrite
        b = 'if __name__ == "__main__":\n     import os\n     foobar()\n'
        f.write(b)
        f.close()
        ssh = paramiko.SSHClient()  # https://blog.ruanbekker.com/blog/2018/04/23/using-paramiko-module-in-python-to-execute-remote-bash-commands/
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect(hostname='3.94.21.39', username='ubuntu', key_filename='test.pem')


        ftp_client=ssh.open_sftp()  # https://medium.com/@keagileageek/paramiko-how-to-ssh-and-file-transfers-with-python-75766179de73
        ftp_client.put('temp.py', "temp.py")
        ftp_client.close()
        
        ssh.connect(hostname='3.94.21.39', username='ubuntu', key_filename='test.pem')

        stdin, stdout, stderr = ssh.exec_command("python3 temp.py")

        for line in stdout.read().splitlines():
            print(line)

        ssh.close()

    return modfied_func  # we can modify func's output

In [190]:
@my_decorator
def foobar():
    print("new")
    print(os.getpid())

In [191]:
foobar()

b'new'
b'4355'
